### Example for retrieving feature store records

Assumptions - Must have a Feature Group Provisioned

Function wrapping get feature store record

In [4]:
def get_fs_record(boto_fs_client, fg_name, feat_id, feat_names=None):
    import time
    
    req = dict(
        FeatureGroupName=fg_name, 
        RecordIdentifierValueAsString=str(feat_id)
    )
    if feat_names:
        req['FeatureNames'] = feat_names
    
    start = time.time()
    record = boto_fs_client.get_record(**req)
    end = time.time()
    duration = end-start
    
    return (record.get('Record', None), duration)

Function to compute avg, 50, 95 and 99 th percentiles

In [5]:
def get_stats(times):
    import numpy as np
    a = np.array(times)
    p_50 = np.percentile(a, 50)
    p_95 = np.percentile(a, 95)
    p_99 = np.percentile(a, 99)
    avg = np.average(a)
    
    return {
        'avg':avg, 
        'p_50': p_50, 
        'p_95': p_95, 
        'p_99': p_99}

Call n times and compute stats on the duration it took for get record from Feature Store

In [6]:
# plain vanilla boto3 session
import boto3
fg_name = 'ingest-fg-05-25-2021-17-24-43'
feat_id = 20000
num_runs = 1000
boto_session = boto3.Session()
boto_fs_client = boto_session.client(
    service_name='sagemaker-featurestore-runtime'
    )

times = []
for i in range(num_runs):
    _, duration = get_fs_record(boto_fs_client, fg_name, feat_id)
    times.append(duration)
    
get_stats(times)

{'avg': 0.011067635536193847,
 'p_50': 0.00900721549987793,
 'p_95': 0.011722838878631587,
 'p_99': 0.079024062156677}

In [7]:
get_stats(times[1:])

{'avg': 0.010869085609733879,
 'p_50': 0.009006977081298828,
 'p_95': 0.011686635017395017,
 'p_99': 0.04814841747283879}